In [1]:
place = 'los angeles'
keyword = 'android developer'

In [2]:
def get_job_description(job_url):
    import requests
    from bs4 import BeautifulSoup
    
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, sdch, br',
        'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
        'referer': 'https://www.glassdoor.com/',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive'
    }
    job_description = ''
    response = requests.post(job_url, headers=headers)
    if response.status_code != 200:
        return job_description
    result = BeautifulSoup(response.content,'lxml')
    if not result.find('div', class_='jobDesc'):
        return job_description
    return result.find('div', class_='jobDesc').get_text()

In [6]:
def get_job_list(place, keyword):
    import requests
    from bs4 import BeautifulSoup
    import time
    
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, sdch, br',
        'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
        'referer': 'https://www.glassdoor.com/',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive'
    }
    
    location_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.01',
        'accept-encoding': 'gzip, deflate, sdch, br',
        'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
        'referer': 'https://www.glassdoor.com/',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive'
    }

    data = {"term": place,
            "maxLocationsToReturn": 10}
    
    location_url = "https://www.glassdoor.co.in/findPopularLocationAjax.htm?"
    
    try:
        # get location id for search location
        location_response = requests.post(location_url, headers=location_headers, data=data).json()
        place_id = location_response[0]['locationId']
        job_list_url = 'https://www.glassdoor.com/Job/jobs.htm'
        absolute_url = 'https://www.glassdoor.com'
        data = {
            'clickSource': 'searchBtn',
            'sc.keyword': keyword,
            'locT': 'C',
            'locId': place_id,
            'jobType': ''
        }
        
        result_list = list()
        
        if place_id:
            for i in range(30):     # don't know why, the website can only show the first 30 pages
                response = requests.post(job_list_url, headers=headers, data=data)
                results_page = BeautifulSoup(response.content,'lxml')
                jobs = results_page.find('', class_='jlGrid hover').find_all('li')
                
                for job in jobs:
                    # get the attributes of the job
                    job_name = job.find('div', class_='flexbox jobTitle').find('a').get_text()
                    job_url = absolute_url+job.find('div', class_='flexbox jobTitle').find('a').get('href')
                    company_location = job.find('div', class_='flexbox empLoc').find('div').get_text().split(' – ')
                    job_company = company_location[0][1:]
                    job_city = company_location[1].split(', ')[0]
                    job_state = company_location[1].split(', ')[1]
                    if job.find('span', class_='green small'):
                        job_salary = job.find('span', class_='green small').get_text()[1:]
                    else:
                        job_salary = ''
                    print(job_url)
                    job_description = get_job_description(job_url)
                    result_list.append((job_name, job_company, job_city, job_state, job_salary, job_description, job_url))
                    
                if not results_page.find('li', class_='next').find('a'):
                    break   # reach the end page
                # next page
                job_list_url = absolute_url + results_page.find('li', class_='next').find('a').get('href')
                time.sleep(2)
                    
            return result_list         
        else:
            print('Location not available.')
              
    except:
        print('Fail to find jobs.')

In [7]:
result = get_job_list(place, keyword)

https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=322198&s=149&guid=000001675d4081869b17242ab0fcf197&src=GD_JOB_AD&t=SRFJ&extid=4&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_ec1afdac&cb=1543457768290&jobListingId=2909234457
https://www.glassdoor.com/partner/jobListing.htm?pos=102&ao=441344&s=149&guid=000001675d4081869b17242ab0fcf197&src=GD_JOB_AD&t=SRFJ&extid=4&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_f7b095c2&cb=1543457768291&jobListingId=2945204971
https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=318376&s=58&guid=000001675d408186b9fc8ef1ee9e532b&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_4cfaf43c&cb=1543457768293&jobListingId=2960185184
https://www.glassdoor.com/partner/jobListing.htm?pos=102&ao=148364&s=58&guid=000001675d408186b9fc8ef1ee9e532b&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_263fe62a&cb=1543457768293&jobListingId=2941018053
https://www.glassdoor.com/partner/jobListing.htm?pos=103&ao=85058&

https://www.glassdoor.com/partner/jobListing.htm?pos=207&ao=85058&s=58&guid=000001675d420eab83626df5cf5ce5ef&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_e22b3117&cb=1543457870116&jobListingId=3021766942
https://www.glassdoor.com/partner/jobListing.htm?pos=208&ao=215203&s=58&guid=000001675d420eab83626df5cf5ce5ef&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_d649b64d&cb=1543457870117&jobListingId=2845594754
https://www.glassdoor.com/partner/jobListing.htm?pos=209&ao=176700&s=58&guid=000001675d420eab83626df5cf5ce5ef&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_154614c6&cb=1543457870118&jobListingId=2840444533
https://www.glassdoor.com/partner/jobListing.htm?pos=210&ao=133516&s=58&guid=000001675d420eab83626df5cf5ce5ef&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_6cc086e1&cb=1543457870119&jobListingId=3003619589
https://www.glassdoor.com/partner/jobListing.htm?pos=211&ao=133801&s=58&g

https://www.glassdoor.com/partner/jobListing.htm?pos=313&ao=302001&s=58&guid=000001675d43624895d378d6d613ab6f&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&aa=1&ea=1&cs=1_c358fd44&cb=1543457956932&jobListingId=2755993720
https://www.glassdoor.com/partner/jobListing.htm?pos=314&ao=395376&s=58&guid=000001675d43624895d378d6d613ab6f&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_99b82797&cb=1543457956933&jobListingId=2950981837
https://www.glassdoor.com/partner/jobListing.htm?pos=315&ao=288527&s=58&guid=000001675d43624895d378d6d613ab6f&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&aa=1&ea=1&cs=1_d3b9c192&cb=1543457956934&jobListingId=3008577651
https://www.glassdoor.com/partner/jobListing.htm?pos=316&ao=389273&s=58&guid=000001675d43624895d378d6d613ab6f&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_0844dc1a&cb=1543457956936&jobListingId=2943179956
https://www.glassdoor.com/partner/jobListing.htm?pos

https://www.glassdoor.com/partner/jobListing.htm?pos=419&ao=374045&s=58&guid=000001675d44436183e7387b2bb7b2c2&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_7581482a&cb=1543458014499&jobListingId=2823866278
https://www.glassdoor.com/partner/jobListing.htm?pos=420&ao=189299&s=58&guid=000001675d44436183e7387b2bb7b2c2&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_fe7cd739&cb=1543458014500&jobListingId=3024590061
https://www.glassdoor.com/partner/jobListing.htm?pos=421&ao=9619&s=58&guid=000001675d44436183e7387b2bb7b2c2&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_cac54f15&cb=1543458014501&jobListingId=2815355136
https://www.glassdoor.com/partner/jobListing.htm?pos=422&ao=322196&s=58&guid=000001675d44436183e7387b2bb7b2c2&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_e42c81d1&cb=1543458014502&jobListingId=2392561219
https://www.glassdoor.com/partner/jobListing.htm?pos=423&ao=291820&s=58&gu

https://www.glassdoor.com/partner/jobListing.htm?pos=525&ao=160415&s=58&guid=000001675d4507c7b7659a0fbb3c19d8&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_fb7db2cd&cb=1543458064858&jobListingId=2886791012
https://www.glassdoor.com/partner/jobListing.htm?pos=526&ao=160415&s=58&guid=000001675d4507c7b7659a0fbb3c19d8&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_2a1abfd1&cb=1543458064859&jobListingId=2886790894
https://www.glassdoor.com/partner/jobListing.htm?pos=527&ao=4341&s=58&guid=000001675d4507c7b7659a0fbb3c19d8&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&aa=1&ea=1&cs=1_d2b80735&cb=1543458064861&jobListingId=2876062880
https://www.glassdoor.com/partner/jobListing.htm?pos=528&ao=280982&s=58&guid=000001675d4507c7b7659a0fbb3c19d8&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_41282922&cb=1543458064862&jobListingId=3033236575
https://www.glassdoor.com/partner/jobListing.htm?pos=529&ao=3728

https://www.glassdoor.com/partner/jobListing.htm?pos=630&ao=291820&s=58&guid=000001675d4604d5a4f0cf8253400963&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_b38aa82c&cb=1543458129611&jobListingId=3039805025
https://www.glassdoor.com/partner/jobListing.htm?pos=701&ao=352789&s=58&guid=000001675d4715b2b33f5644049b75b4&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_750e4fb5&cb=1543458199456&jobListingId=3031265017
https://www.glassdoor.com/partner/jobListing.htm?pos=702&ao=4120&s=58&guid=000001675d4715b2b33f5644049b75b4&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_ce3b2fbc&cb=1543458199458&jobListingId=2916776322
https://www.glassdoor.com/partner/jobListing.htm?pos=703&ao=140152&s=58&guid=000001675d4715b2b33f5644049b75b4&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_7363dd5e&cb=1543458199459&jobListingId=2930848695
https://www.glassdoor.com/partner/jobListing.htm?pos=704&ao=133118&s=58&gu

https://www.glassdoor.com/partner/jobListing.htm?pos=805&ao=352789&s=58&guid=000001675d47fd749fc480c970b946b6&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_e70695ff&cb=1543458258566&jobListingId=3033744122
https://www.glassdoor.com/partner/jobListing.htm?pos=806&ao=352789&s=58&guid=000001675d47fd749fc480c970b946b6&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_7d5447aa&cb=1543458258568&jobListingId=3040406491
https://www.glassdoor.com/partner/jobListing.htm?pos=807&ao=389480&s=58&guid=000001675d47fd749fc480c970b946b6&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_11e7fa93&cb=1543458258569&jobListingId=3036109612
https://www.glassdoor.com/partner/jobListing.htm?pos=808&ao=182759&s=58&guid=000001675d47fd749fc480c970b946b6&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_fedcc9d9&cb=1543458258571&jobListingId=3040277224
https://www.glassdoor.com/partner/jobListing.htm?pos=809&ao=193811&s=58&